In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import pyperclip
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import requests as req
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
  # 웹드라이버 실행
driver = webdriver.Chrome()

# 제품 목록 페이지 로드
driver.get("https://www.flipkart.com/air-conditioners/pr?sid=j9e%2Cabm%2Cc54&p%5B%5D=facets.brand%255B%255D%3DSAMSUNG&otracker=categorytree&otracker=nmenu_sub_TVs+%26+Appliances_0_LG")

# 페이지 로딩 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'CGtC98')))

# URL을 저장할 리스트 초기화
url_list = []

# 요소 찾기
elements = driver.find_elements(By.CLASS_NAME, 'CGtC98')

# 각 요소에서 'href' 속성을 가져와 리스트에 추가 (sponsored 요소는 제외)
for element in elements:
    try:
        # 'f8qK5m' 클래스를 가진 자식 요소가 있는지 확인
        if element.find_elements(By.CLASS_NAME, 'f8qK5m'):
            # 'f8qK5m' 클래스를 가진 자식 요소가 있다면 skip
            continue
        # 자식 요소가 'f8qK5m' 클래스를 가지고 있지 않다면 href 추출
        url_value = element.get_attribute('href')
        if url_value:  # href 속성이 있는 경우만 추가
            url_list.append(url_value)
    except NoSuchElementException:
        # 자식 요소가 없으면 그냥 넘어가기
        continue 
        
# 리뷰 데이터를 저장할 리스트 초기화
all_reviews = []

for url in tqdm(url_list):
    try:
        driver.get(url)
        
        # XPath를 사용하여 제목 찾기. 제품 코드 별 XPATH 달라서 접근 불가... 제목에서 코드 추출하세용...
        try:
            # XPath를 사용하여 제목 찾기. 제품 코드 별 XPATH 달라서 접근 불가... 제목에서 코드 추출하세용... 
            list_items = driver.find_elements(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[2]/div/div[1]/h1/span')
            
            # 텍스트 추출
            hp_data = [item.text for item in list_items]
            if not hp_data:
                hp_data = ["No data"]  # 데이터가 없을 경우 기본값
        except (NoSuchElementException, TimeoutException) as e:
            hp_data = ["Error fetching data"]  # 오류 발생 시 기본값
        
        # 리뷰 버튼 클릭 대기
        try:
            review_button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "_23J90q.RcXBOT")))
            if review_button.is_displayed():
                review_button.click()
        except (NoSuchElementException, TimeoutException):
            pass
        
        # 낮은 별점 접근
        try:
            low_rating_option = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#container > div > div.VLIitu > div > div > div.DOjaWF.gdgoEp.col-9-12 > div.DOjaWF.gdgoEp.col-12-12 > div > div:nth-child(2) > div > div > div > div > select > option:nth-child(4)")))
            low_rating_option.click()
        except (NoSuchElementException, TimeoutException) as e:
            print(f"Error selecting sort option for URL {url}: {e}")
        
        # 데이터 확보
        page_number = 1
        
        while True:
            try:
                # 페이지 URL 업데이트
                current_url = driver.current_url
                full_url = f'{current_url}&page={page_number}'
                driver.get(full_url)
                
                # ZmyHeo 클래스의 존재 여부를 확인
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ZmyHeo')))
                
                # 데이터 추출
                text_elements = driver.find_elements(By.CLASS_NAME, 'ZmyHeo')
                state_elements = driver.find_elements(By.CLASS_NAME, 'MztJPv')
                date_elements = driver.find_elements(By.CLASS_NAME, '_2NsDsF')
                star_elements = driver.find_elements(By.XPATH, "//*[contains(@class, 'XQDdHH') and (contains(@class, 'Js30Fc') or contains(@class, 'Czs3gR') or contains(@class, 'Ga3i8K'))]")
                
                text_list = [elem.text for elem in text_elements]
                state_list = [elem.text for elem in state_elements]
                date_list = [elem.text for elem in date_elements]
                dates = date_list[1::2]
                star_list = [elem.text for elem in star_elements]
                
                # 리뷰 데이터를 모든 리뷰 리스트에 추가
                for t, s, d, r in zip(text_list, state_list, dates, star_list):
                    for data in hp_data:
                        all_reviews.append({
                            'text': t,
                            'state': s,
                            'date': d,
                            'star': r,
                            'hp_data': data  # 각각의 hp_data 항목을 추가
                        })

                # 페이지 번호 증가
                page_number += 1
                
            except (NoSuchElementException, TimeoutException) as e:
                print(f"Error on page {page_number} for URL {full_url}: {e}")
                break
    
    except Exception as e:
        print(f"Error processing URL {url}: {e}")

# 현재 창 닫기
driver.quit()

  4%|███▍                                                                               | 1/24 [00:30<11:42, 30.53s/it]

Error processing URL https://www.flipkart.com/samsung-ai-pro-1-5-ton-5-star-split-inverter-ac-wi-fi-connect-white/p/itm126bd51087a93?pid=ACNGWH7Y5ASY8ZTC&lid=LSTACNGWH7Y5ASY8ZTCOI4AWU&marketplace=FLIPKART&store=j9e%2Fabm%2Fc54&srno=b_1_1&otracker=browse&iid=599cddd0-2fac-4ea0-b0ba-e5638e7b9b1b.ACNGWH7Y5ASY8ZTC.SEARCH&ssid=8y2a3xvl8w0000001724716334551: name 'NoSuchElementException' is not defined


  8%|██████▉                                                                            | 2/24 [01:00<11:02, 30.12s/it]

Error processing URL https://www.flipkart.com/samsung-convertible-5-in-1-cooling-2024-model-1-5-ton-3-star-split-inverter-ac-white/p/itmdac76fb7a5814?pid=ACNGHRDQZUD2CPTZ&lid=LSTACNGHRDQZUD2CPTZI1ZDMQ&marketplace=FLIPKART&store=j9e%2Fabm%2Fc54&spotlightTagId=FkPickId_j9e%2Fabm%2Fc54&srno=b_1_2&otracker=browse&iid=599cddd0-2fac-4ea0-b0ba-e5638e7b9b1b.ACNGHRDQZUD2CPTZ.SEARCH&ssid=8y2a3xvl8w0000001724716334551: name 'NoSuchElementException' is not defined


 12%|██████████▍                                                                        | 3/24 [01:29<10:24, 29.75s/it]

...

Error processing URL https://www.flipkart.com/samsung-convertible-2-ton-3-star-split-inverter-ac-white/p/itm42f367b6a3470?pid=ACNGGXDFJEYCMAGF&lid=LSTACNGGXDFJEYCMAGFKVPT32&marketplace=FLIPKART&store=j9e%2Fabm%2Fc54&srno=b_1_22&otracker=browse&iid=599cddd0-2fac-4ea0-b0ba-e5638e7b9b1b.ACNGGXDFJEYCMAGF.SEARCH&ssid=8y2a3xvl8w0000001724716334551: name 'NoSuchElementException' is not defined


 96%|██████████████████████████████████████████████████████████████████████████████▌   | 23/24 [10:36<00:24, 24.14s/it]

Error processing URL https://www.flipkart.com/samsung-2022-model-1-5-ton-3-star-split-inverter-ac-white/p/itm973780e7a726d?pid=ACNGMFFP8H3N2H8M&lid=LSTACNGMFFP8H3N2H8MFUHNOC&marketplace=FLIPKART&store=j9e%2Fabm%2Fc54&srno=b_1_23&otracker=browse&iid=599cddd0-2fac-4ea0-b0ba-e5638e7b9b1b.ACNGMFFP8H3N2H8M.SEARCH&ssid=8y2a3xvl8w0000001724716334551: name 'NoSuchElementException' is not defined


100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [11:14<00:00, 28.12s/it]

Error processing URL https://www.flipkart.com/samsung-convertible-5-in-1-cooling-2023-model-1-5-ton-5-star-split-inverter-windfree-technology-ac-wi-fi-connect-white/p/itm4a1a7eb567864?pid=ACNGHRDQTYCKFWZV&lid=LSTACNGHRDQTYCKFWZVQF2X7L&marketplace=FLIPKART&store=j9e%2Fabm%2Fc54&srno=b_1_24&otracker=browse&iid=599cddd0-2fac-4ea0-b0ba-e5638e7b9b1b.ACNGHRDQTYCKFWZV.SEARCH&ssid=8y2a3xvl8w0000001724716334551: name 'NoSuchElementException' is not defined


In [3]:
df_SS_airconditioner = pd.DataFrame(all_reviews)
df_SS_airconditioner

,text,state,date,star,hp_data
0,"This ac have big issue, when you turn on the a...","Certified Buyer, Udgir",1 month ago,1,SAMSUNG AI Pro 1.5 Ton 5 Star Split Inverter A...
1,3rd class ac..\nAir flow quality very poor,"Certified Buyer, Durg",2 months ago,1,SAMSUNG AI Pro 1.5 Ton 5 Star Split Inverter A...
2,The product is very bad its not cooling the sp...,"Certified Buyer, Pataudi",2 months ago,1,SAMSUNG AI Pro 1.5 Ton 5 Star Split Inverter A...
3,Not too good it consume 1 unit in half and hou...,"Certified Buyer, Ganaur",2 months ago,1,SAMSUNG AI Pro 1.5 Ton 5 Star Split Inverter A...
4,One of the worst ACs is never buy Samsung AC. ...,"Certified Buyer, Badlapur",2 months ago,1,SAMSUNG AI Pro 1.5 Ton 5 Star Split Inverter A...
...,...,...,...,...,...
1870,This is one of the best products on a budget. ...,"Certified Buyer, Vasai Virar","Apr, 2023",5,SAMSUNG Convertible 5-in-1 Cooling 2023 Model ...
1871,The best Quality,"Certified Buyer, Bhilai","Mar, 2023",5,SAMSUNG Convertible 5-in-1 Cooling 2023 Model ...
1872,cool 👍 best in budget,"Certified Buyer, Nalgonda","Mar, 2023",5,SAMSUNG Convertible 5-in-1 Cooling 2023 Model ...
1873,Good,"Certified Buyer, Bankura","Mar, 2023",5,SAMSUNG Convertible 5-in-1 Cooling 2023 Model ...


In [4]:
df_SS_airconditioner.to_csv("df_SS_airconditioner.csv")